In [ ]:
import locale
import numpy as np
import os
import pandas as pd
import tempfile
from datetime import datetime
from fdfgen import forge_fdf
from pathlib import Path
from tqdm.notebook import tqdm

# 👇 Configuración

In [ ]:
CURSO = 'CF GA'                       # 1ESO, 2ESO, 3ESO, 3PMAR, 4ESO, 1BACH, 2BACH, FPB SA, CF AD, CF GA, CF COM
DESDE = datetime(2020, 5, 17,  0,  0) # año, mes, día, h, m (INCLUSIVE)
HASTA = datetime(2021, 6, 18, 23, 59) # año, mes, día, h, m (INCLUSIVE)

# 👆

In [ ]:
pdfs_d = {
    '1ESO': '1ESO/02._MATRICULA_ESO_1_LOMCE.pdf',
    '2ESO': '2ESO/02._MATRICULA_ESO_2_LOMCE.pdf',
    '3ESO': '3ESO/02._MATRICULA_ESO_3_LOMCE.pdf',
    '3PMAR': '3PMAR/02._MATRICULA_ESO_3_PMAR_LOMCE.pdf',
    '4ESO': '4ESO/02._MATRICULA_ESO_4_LOMCE.pdf',
    '1BACH': '1BACH/03. MATRÍCULA BAC_1_LOMCE.pdf',
    '2BACH': '2BACH/03. MATRICULA BAC_2_LOMCE.pdf',
    'FPB SA': 'FPB SA/02. MATRÍCULA_FPB SERVICIOS ADMINISTRATIVOS.pdf',
    'CF AD': 'CF AD/02. MATRÍCULA _CF ASISTENCIA A LA DIRECCIÓN.pdf',
    'CF GA': 'CF GA/02. MATRÍCULA _CF GESTIÓN ADMINISTRATIVA.pdf',
    'CF COM': 'CF COM/02. MATRÍCULA _CF ACTIVIDADES COMERCIALES.pdf',
}

csvs_d = {
    '1ESO': '1ESO/1ESO Cuestionario de matrícula.csv',
    '2ESO': '2ESO/2ESO Cuestionario de matrícula.csv',
    '3ESO': '3ESO/3ESO Cuestionario de matrícula.csv',
    '3PMAR': '3PMAR/3PMAR Cuestionario de matrícula.csv',
    '4ESO': '4ESO/4ESO Cuestionario de matrícula.csv',
    '1BACH': '1BACH/1BACH Cuestionario de matrícula.csv',
    '2BACH': '2BACH/2BACH Cuestionario de matrícula.csv',
    'FPB SA': 'FPB SA/FPB SERVICIOS ADMINISTRATIVOS Cuestionario de matrícula.csv',
    'CF AD': 'CF AD/CFGS ASISTENCIA A DIRECCIÓN Cuestionario de matrícula.csv',
    'CF GA': 'CF GA/CFGM GESTIÓN ADMINISTRATIVA Cuestionario de matrícula.csv',
    'CF COM': 'CF COM/CFGM ACTIVIDADES COMERCIALES Cuestionario de matrícula.csv',
}

In [ ]:
csv_file = csvs_d[CURSO]
pdf_file = pdfs_d[CURSO]
out_dir = Path(f'SALIDA_{CURSO}')

In [ ]:
out_dir.mkdir(parents=True, exist_ok=True)

In [ ]:
# lee CSV
mat_df = pd.read_csv(csv_file)

In [ ]:
# traduce fecha moodle -> datetime
locale.setlocale(locale.LC_TIME, 'es_ES')
mat_df['Fecha'] = mat_df['Fecha'].apply(lambda x: datetime.strptime(x, '%A, %d de %B de %Y, %H:%M'))

In [ ]:
# filtra fuera de fechas
mat_df = mat_df[(mat_df['Fecha'] >= DESDE) & (mat_df['Fecha'] <= HASTA)]
print(f'Hay {len(mat_df)} matrículas')

In [ ]:
# nulos -> '---'
mat_df = mat_df.fillna('---')

In [ ]:
# multi-respuestas -> listas
if CURSO[:2] == 'CF':
    mat_df['mods'] = mat_df['Módulos formativos de 1º'].str.split('\n') + mat_df['Módulos formativos de 2º'].str.split('\n')

In [ ]:
# Imprime la 1ª matrícula
mat_df[:1].T

In [ ]:
# optativas
opts = {
    'Alemán 2º idioma': 'ALE', # 4ESO
    'Anatomía Aplicada': 'AA', # 1BAT
    'Competencia Comunicativa Oral en Inglés': 'CCOI',
    'Cultura Científica': 'CCIE', # 4ESO
    'Cultura Clásica': 'CCLA',
    'Educación Plástica': 'EPV', # 4ESO
    'Educación Plástica, Visual y Audiovisual': 'EPVA',
    'Filosofía': 'FIL', # 4ESO
    'Física': 'FIS',
    'Francés 2º idioma': 'FRA', # 4ESO
    'Geología': 'GEOL',
    'Informática': 'INF', # 4ESO
    'Iniciación a la Actividad Emprendedora y Empresarial': 'IAEE',
    'Música': 'MUS', # 4ESO
    'Proyecto Interdisciplinar: Impresión 3D, programación y robótica aplicadas': 'PI', # 4ESO
    'Psicología': 'PSI',
    'Religión Católica': 'REL', # 1BAT, 2BAT
    'Taller de refuerzo de Castellano': 'RCAS',
    'Taller de refuerzo de Matemáticas': 'RMAT',
    'Tecnología': 'TEC', # 3ESO, 4ESO
    'Tecnologías de la Información y la Comunicación I': 'TIC1', # 1BAT
    'Tecnologías de la Información y la Comunicación II': 'TIC2', # 1BAT
}

In [ ]:
# módulos
mods = {
    'FPB SA': {
        # 1º FPB SA
        '3001. Tratamiento Informático de Datos': 'fpb1_tid',
        '3003. Técnicas Administrativas Básicas': 'fpb1_tab',
        '3005. Atención al Cliente': 'fpb1_ac',
        '3006. Preparación de Pedidos y Venta de Productos': 'fpb1_ppvp',
        '3009. Ciencias Aplicadas I': 'fpb1_ccaa1',
        '3011. Comunicación y Sociedad I': 'fpb1_cys1',
        'CV0005. Formación y Orientación Laboral I': 'fpb1_fol1',
        # 2º FPB SA
        '3002. Aplicaciones Básicas de Ofimática': 'fpb2_abo',
        '3004. Archivo y Comunicación': 'fpb2_ac',
        '3010. Ciencias Aplicadas II': 'fpb2_ccaa2',
        '3012. Comunicación y Sociedad II': 'fpb2_cys2',
        'CV0006 Formación y Orientación Laboral II': 'fpb2_fol2',
        '3008. Formación en Centros de Trabajo': 'fpb2_fct',
    },

    'CF AD': {
        # 1º
        '0647. Gestión de la documentación jurídica y empresarial': '1ad_gdje',
        '0648. Recursos humanos y responsabilidad social corporativa': '1ad_rhrsc',
        '0649. Ofimática y proceso de la información': '1ad_opi',
        '0650. Proceso integral de la actividad comercial': '1ad_piac',
        '0651. Comunicación y atención al cliente': '1ad_cac',
        '0179. Inglés': '1ad_ing',
        '0665. Formación y orientación laboral': '1ad_fol',
        # 2º
        '0661. Protocolo empresarial': '2ad_pe',
        '0662. Organización de eventos empresariales': '2ad_oee',
        '0663. Gestión avanzada de la información': '2ad_gai',
        '0180. Segunda lengua extranjera': '2ad_fr2',
        'CV0004. Inglés técnico II-S / Horario reserv. docencia en inglés': '2ad_ingtec',
        '0664. Proyecto de asistencia a la dirección': '2ad_proyecto',
        '0667. Formación en centros de trabajo': '2ad_fct',
    },
    
    'CF GA': {
        # 1º
        '0437. Comunicación empresarial y atención al cliente': '1ga_ceac',
        '0438. Operaciones administrativas de compraventa': '1ga_oacv',
        '0439. Empresa y administración': '1ga_ea',
        '0440. Tratamiento informático de la información': '1ga_tii',
        '0441. Técnica contable': '1ga_tc',
        '0156. Inglés': '1ga_ing',
        '0449. Formación y orientación laboral': '1ga_fol',
        
        # 2º
        '0442. Operaciones administrativas de compraventa': '2ga_oarh',
        '0443. Tratamiento de la documentación contable': '2ga_tdc',
        '0446. Empresa en el aula': '2ga_eea',
        '0448. Operaciones auxiliares de gestión de tesorería': '2ga_oagt',
        'CV0002. Inglés técnico II-M / Horario reservado docencia en inglés': '2ga_ingtec',
        '0451. Formación en centros de trabajo': '2ga_fct',
    },
    
    'CF COM': {
        #1º
        '1226. Marketing en la actividad comercial': '1ac_mac',
        '1229. Gestión de compras': '1ac_gc',
        '1231. Dinamización del punto de venta': '1ac_dpb',
        '1232. Procesos de venta': '1ac_pv',
        '1233. Aplicaciones informáticas para el comercio': '1ac_aic',
        '0156. Inglés': '1ac_ing',
        '1236. Formación y Orientación Laboral': '1ac_fol',
        # 2º
        '1227. Gestión de un pequeño comercio': '2ac_gpc',
        '1228. Técnicas de almacén': '2ac_ta',
        '1230. Venta técnica': '2ac_vt',
        '1234. Servicios de atención comercial': '2ac_sac',
        '1235. Comercio electrónico': '2ac_ce',
        'CV0002. Inglés técnico II-M/Horario reserv. docencia Inglés': '2ac_intec',
        '1237. Formación en Centros de Trabajo': '2ac_fct',
    }
}

In [ ]:
# Separa nombre de apellidos
na = mat_df['Nombre completo del usuario'].str.split(', ', expand=True)
mat_df['apellidos_alu'] = na[0]
mat_df['nombre_alu'] = na[1]

# Rellena los checkboxes
mat_df['convive_padre'] = np.where(mat_df['Convive con'] == 'Padre', 'Yes', 'Off')
mat_df['convive_madre'] = np.where(mat_df['Convive con'] == 'Madre', 'Yes', 'Off')
mat_df['convive_ambos'] = np.where(mat_df['Convive con'] == 'Ambos', 'Yes', 'Off')

# Todas las matrículas vía aules son de alumnos del centro
mat_df['centro_procedencia'] = 'IES Nº1 LIBERTAS'

# Junta apellido + nombre de padre/madre
mat_df['nombre_pad'] = mat_df['Apellidos del padre o tutor legal'] + ', ' + mat_df['Nombre del padre o tutor legal']
mat_df['nombre_mad'] = mat_df['Apellidos de la madre o tutora legal'] + ', ' + mat_df['Nombre de la madre o tutora legal']

# Renombra columnas csv -> campos pdf
mat_df = mat_df.rename(columns={
    'Con quién convive el alumno': 'convive_otros', 
    'Domicilio (calle/plaza y número) del alumno': 'domicilio_alu',
    'Código postal del alumno<span class="boundaries"></span>': 'cp_alu',
    'Localidad del alumno': 'localidad_alu',
    'Provincia del alumno': 'provincia_alu',

    'DNI/NIE del padre o tutor legal': 'dni_pad',
    'E-MAIL del padre o tutor legal': 'email_pad',
    'Teléfono 1 del padre o tutor legal': 'telefono1_pad',
    'Teléfono 2 del padre o tutor legal': 'telefono2_pad',
    'Domicilio (calle/plaza y número) del padre o tutor legal': 'domicilio_pad',
    'Código postal del padre o tutor legal<span class="boundaries"></span>': 'cp_pad',
    'Localidad del padre o tutor legal': 'localidad_pad',
    'Provincia del padre o tutor legal': 'provincia_pad',

    'DNI/NIE de la madre o tutora legal': 'dni_mad',
    'E-MAIL de la madre o tutora legal': 'email_mad',
    'Teléfono 1 de la madre o tutora legal': 'telefono1_mad',
    'Teléfono 2 de la madre o tutora legal': 'telefono2_mad',
    'Domicilio (calle/plaza y número) de la madre o tutora legal': 'domicilio_mad',
    'Código postal de la madre o tutora legal<span class="boundaries"></span>': 'cp_mad',
    'Localidad de la madre o tutora legal': 'localidad_mad',
    'Provincia de la madre o tutora legal': 'provincia_mad',

    'Grupo actual': 'grupo',
})

## Casillas especificas por curso (ESO)

In [ ]:
if CURSO in ['1ESO', '2ESO', '3ESO', '4ESO']:
    mat_df['pluri_si'] = np.where(mat_df['Desea pertenecer al grupo plurilingüe'] == 'Sí', 'Yes', 'Off')
    mat_df['pluri_no'] = np.where(mat_df['Desea pertenecer al grupo plurilingüe'] == 'No', 'Yes', 'Off')

if CURSO in ['3ESO']:
    mat_df['MAA'] = np.where(mat_df['Elija Matemáticas'] == 'Académicas', 'Yes', 'Off')
    mat_df['MAP'] = np.where(mat_df['Elija Matemáticas'] == 'Aplicadas', 'Yes', 'Off')
    
if CURSO in ['4ESO']:
    mat_df['itinerario_ciencias'] = np.where(mat_df['Elija itinerario'] == 'Bachillerato Ciencias', 'Yes', 'Off')
    mat_df['itinerario_humanidades'] = np.where(mat_df['Elija itinerario'] == 'Bachillerato Humanidades o CCSS', 'Yes', 'Off')
    mat_df['itinerario_aplicadas'] = np.where(
        (mat_df['Elija itinerario'] == 'Enseñanzas aplicadas con Ciencias Aplicadas a la Actividad Profesional') |
        (mat_df['Elija itinerario'] == 'Enseñanzas aplicadas con Iniciación a la Actividad Emprendedora y Empresarial'),
        'Yes', 'Off'
    )
    mat_df['itinerario_aplicadas_cacp'] = np.where(
        mat_df['Elija itinerario'] == 'Enseñanzas aplicadas con Ciencias Aplicadas a la Actividad Profesional',
        'X', ''
    )
    mat_df['itinerario_aplicadas_iaee'] = np.where(
        mat_df['Elija itinerario'] == 'Enseñanzas aplicadas con Iniciación a la Actividad Emprendedora y Empresarial',
        'X', ''
    )

if CURSO in ['1ESO', '2ESO', '3ESO', '3PMAR', '4ESO']:
    mat_df['religion'] = np.where(mat_df['Elija Religión o Valores Éticos'] == 'Religión', 'Yes', 'Off')
    mat_df['valores']  = np.where(mat_df['Elija Religión o Valores Éticos'] == 'Valores Éticos', 'Yes', 'Off')
    mat_df['croni_si'] = np.where(mat_df['El alumno padece enfermedad crónica'] == 'Sí', 'Yes', 'Off')
    mat_df['croni_no'] = np.where(mat_df['El alumno padece enfermedad crónica'] == 'No', 'Yes', 'Off')
    mat_df['ampa_si']  = np.where(mat_df['Pertenece a la AMPA'] == 'Sí', 'Yes', 'Off')
    mat_df['ampa_no']  = np.where(mat_df['Pertenece a la AMPA'] == 'No', 'Yes', 'Off')

## Casillas especificas por curso (1BACH)

In [ ]:
if CURSO in ['1BACH']:
    # Itinerarios
    mat_df['itinerario_ciencias'] = np.where(
        (mat_df['Elija itinerario'] == 'Ciencias con Dibujo Técnico I') |
        (mat_df['Elija itinerario'] == 'Ciencias con Biología y Geología'),
        'Yes', 'Off'
    )
    mat_df['itinerario_humanidades'] = np.where(
        (mat_df['Elija itinerario'] == 'Humanidades con Hª Mundo Contemporáneo') |
        (mat_df['Elija itinerario'] == 'Humanidades con Literatura Universal'),
        'Yes', 'Off'
    )
    mat_df['itinerario_ccss'] = np.where(
        (mat_df['Elija itinerario'] == 'CCSS con Hª Mundo Contemporáneo') |
        (mat_df['Elija itinerario'] == 'CCSS con Literatura Universal'),
        'Yes', 'Off'
    )
    # Troncales
    mat_df['troncal_DT1'] = np.where(
        mat_df['Elija itinerario'] == 'Ciencias con Dibujo Técnico I',
        'X', ''
    )
    mat_df['troncal_BG'] = np.where(
        mat_df['Elija itinerario'] == 'Ciencias con Biología y Geología',
        'X', ''
    )
    mat_df['troncal_HMC'] = np.where(
        (mat_df['Elija itinerario'] == 'Humanidades con Hª Mundo Contemporáneo') |
        (mat_df['Elija itinerario'] == 'CCSS con Hª Mundo Contemporáneo'),
        'X', ''
    )
    mat_df['troncal_LU'] = np.where(
        (mat_df['Elija itinerario'] == 'Humanidades con Literatura Universal') |
        (mat_df['Elija itinerario'] == 'CCSS con Literatura Universal'),
        'X', ''
    )

## Casillas especificas por curso (2BACH)

In [ ]:
if CURSO in ['2BACH']:
    # Itinerarios
    mat_df['itinerario_ciencias'] = np.where(
        (mat_df['Elija itinerario'] == 'Ciencias + Matemáticas II + Biología + Química') |
        (mat_df['Elija itinerario'] == 'Ciencias + Matemáticas II + Física + Dibujo Técnico II') |
        (mat_df['Elija itinerario'] == 'Ciencias + Matemáticas II + Física + Química'),
        'Yes', 'Off'
    )
    mat_df['itinerario_humanidades'] = np.where(
        (mat_df['Elija itinerario'] == 'Humanidades + Geografía') |
        (mat_df['Elija itinerario'] == 'Humanidades + Hª del Arte'),
        'Yes', 'Off'
    )
    mat_df['itinerario_ccss'] = np.where(
        (mat_df['Elija itinerario'] == 'CCSS + Geografía') |
        (mat_df['Elija itinerario'] == 'CCSS + Hª del Arte'),
        'Yes', 'Off'
    )
    # Troncales
    mat_df['troncal_mat2_bio_qui'] = np.where(
        mat_df['Elija itinerario'] == 'Ciencias + Matemáticas II + Biología + Química',
        'X', ''
    )
    mat_df['troncal_mat2_fis_dt2'] = np.where(
        mat_df['Elija itinerario'] == 'Ciencias + Matemáticas II + Física + Dibujo Técnico II',
        'X', ''
    )
    mat_df['troncal_mat2_fis_qui'] = np.where(
        mat_df['Elija itinerario'] == 'Ciencias + Matemáticas II + Física + Química',
        'X', ''
    )
    mat_df['troncal_GEO'] = np.where(
        (mat_df['Elija itinerario'] == 'Humanidades + Geografía') |
        (mat_df['Elija itinerario'] == 'CCSS + Geografía'),
        'X', ''
    )
    mat_df['troncal_HA'] = np.where(
        (mat_df['Elija itinerario'] == 'Humanidades + Hª del Arte') |
        (mat_df['Elija itinerario'] == 'CCSS + Hª del Arte'),
        'X', ''
    )

    # EF voluntaria
    mat_df['vol_ef'] = np.where(mat_df['Cursar Educación Fisicodeportiva y Salud'] == 'Sí', 'X', '')

## Casillas específicas ciclos

In [ ]:
def cumplimenta_checkboxes(mat_df, pregunta, respuestas_y_campos):
    for respuesta, campo in respuestas_y_campos.items():
        mat_df[campo] = np.where(mat_df[pregunta] == respuesta, 'Yes', 'Off')

In [ ]:
if CURSO == 'CF AD':
    cumplimenta_checkboxes(
        mat_df,
        'Accede al ciclo por cumplir el siguiente requisito',
        {
            'Superar prueba de acceso a un ciclo formativo de grado superior' : 'acceso_pac',
            'Superar prueba de acceso a la universidad para mayores de 25 años' : 'acceso_25',
            'Título de Bachiller LOGSE/LOCE/LOE/COU/BUP' : 'acceso_batx',
            'Acceso desde un grado medio' : 'acceso_gm',
        }
    )

In [ ]:
if CURSO in ['CF GA', 'CF COM']:
    cumplimenta_checkboxes(
        mat_df,
        'Accede al ciclo por cumplir el siguiente requisito',
        {
            'Superar prueba de acceso a un ciclo formativo de grado medio' : 'acceso_pac',
            'Superar prueba de acceso a la universidad para mayores de 25 años' : 'acceso_25',
            'Haber superado 2º BUP' : 'acceso_2bup',
            'Título Profesional Básico' : 'acceso_tpb',
            'Título de Bachiller LOGSE/LOCE/LOE/LOMCE' : 'acceso_batx',
            'Graduado en E.S.O.' : 'acceso_ges',
        }
    )

## Numera optativas por preferencia

In [ ]:
for nbloque, enunciado in enumerate(['Optativa', 'Primera optativa', 'Segunda optativa']):
    for k, v in opts.items():
        conds = [ mat_df[f'{enunciado} ({x}ª preferencia)'] == k for x in range(1, 10) if f'{enunciado} ({x}ª preferencia)' in mat_df.columns ]
        choices = list(range(1, len(conds) + 1))
        if choices:
            mat_df[f'O{nbloque}_{v}'] = np.select(conds, choices, default='')

## Módulos FP

In [ ]:
if CURSO in ['CF AD', 'CF GA', 'CF COM']:
    for nombre_modulo, campo_pdf in mods[CURSO].items():
        seleccion = mat_df['mods'].apply(lambda respuesta: nombre_modulo in respuesta)
        mat_df[campo_pdf] = np.where(seleccion, 'Yes', 'Off')

## Genera PDFs

In [ ]:
for _, row in tqdm(mat_df.iterrows(), total=len(mat_df)):
    nombre = row['Nombre completo del usuario']
    fields = [ (k, v) for k, v in row.to_dict().items() ]
    #print(fields)
    fdf = forge_fdf("", fields, [], [], [])
    tmp_file = 'tmp.fdf'
    with open(tmp_file, 'wb') as fdf_file:
        fdf_file.write(fdf)

    if CURSO in ['1ESO', '2ESO', '3ESO', '3PMAR', '4ESO', '1BACH', '2BACH']:
        grupo  = row['grupo']
        output_file = out_dir / f'{grupo} {nombre}.pdf'
    else:
        output_file = out_dir / f'{nombre}.pdf'
        
    cmd = f'pdftk "{pdf_file}" fill_form "{tmp_file}" output "{output_file}" dont_ask'
    
    assert os.system(cmd) == 0, f'Petada en la generación del PDF, mira el log de jupyter:\n{cmd}'
    os.remove(tmp_file)